In [11]:
from jax import numpy as jnp, random as jr
from jax import tree_util as jtu
from jax import nn, lax

from pymdp.agent import Agent
from pymdp.envs import TMaze

from pybefit.inference import (
    run_nuts,
    run_svi,
    default_dict_nuts,
    default_dict_numpyro_svi,
)

from pybefit.inference import NumpyroModel, NumpyroGuide
from pybefit.inference import Normal, NormalPosterior
from pybefit.inference.numpyro.likelihoods import pymdp_likelihood as likelihood
from numpyro.infer import Predictive


In [12]:
seed_key = jr.PRNGKey(101)

# setting the parameters for the environment
batch_size = 100 # batch_size, which in this case corresponds to the number of agents to fit in parallel
num_blocks = 25
num_timesteps = 5

reward_condition = None # 0 is reward in left arm, 1 is reward in right arm, None is random allocation
reward_probability = 0.9 # 100% chance of reward in the correct arm
punishment_probability = 0.8 # 100% chance of punishment in the other arm
cue_validity = 1.0 # 100% valid cues
dependent_outcomes = False # if True, punishment occurs as a function of reward probability (i.e., if reward probability is 0.8, then 20% punishment). If False, punishment occurs with set probability (i.e., 20% no outcome and punishment will only occur in the other (non-rewarding) arm)

# initialising the environment. see tmaze.py in pymdp/envs for the implementation details.
task = TMaze( 
    batch_size=batch_size, 
    reward_probability=reward_probability,     
    punishment_probability=punishment_probability, 
    cue_validity=cue_validity,          
    reward_condition=reward_condition,
    dependent_outcomes=dependent_outcomes
)

key, _key = jr.split(seed_key)
_, task = task.reset(jr.split(_key, batch_size))

In [13]:
#  setting A tensors from the environment parameters
A = task.params["A"]
A_dependencies = task.dependencies["A"]  # dependencies allow you to specify which state factors each observation modality depends on, so you dont have to store all the conditional dependencies between all state factors and each modality

# setting B tensors from the environment parameters
B = task.params["B"]
B_dependencies = task.dependencies["B"]  # dependencies allow you to specify which state factors each state factor depends on, so you dont have to store all the conditional dependencies between all state factors
 
# creating C tensors filled with zeros for [location], [reward], [cue] based on A shapes
C_reward = jnp.array([0., 2., -3.])  # prefer reward (index 1), avoid punishment (index 2)
C = [jnp.zeros((batch_size, a.shape[1]), dtype=jnp.float32) for a in A]
# setting preferences for outcomes only
C[1] = jnp.broadcast_to(C_reward, (batch_size, 3))

# creating D tensors [location], [reward] based on B shapes
D = [jnp.zeros((batch_size, b.shape[1]), dtype=jnp.float32) for b in B]
# D[0]: location - all zeros except location 0 (center) because the agent always starts in the center
D_loc = jnp.zeros((batch_size, B[0].shape[1]), dtype=jnp.float32) 
D_loc = D_loc.at[:,0].set(1.0)  # set center location to 1.0
D[0] = D_loc

# D[1]: reward location - uniform distribution
D_reward = jnp.ones((batch_size, B[1].shape[1]), dtype=jnp.float32) 
D_reward = D_reward / jnp.sum(D_reward, axis=1, keepdims=True)  # normalize to get uniform distribution
D[1] = D_reward

In [14]:
agents = Agent(
    A, B, C=C, D=D, A_dependencies=A_dependencies, B_dependencies=B_dependencies, batch_size=batch_size
)

model_dims = agents.get_model_dimensions()  # get model dimensions for useful downstream applications

In [15]:
# test env step method
keys = jr.split(key, batch_size)
obs, task = task.step(keys)

actions = jnp.zeros((batch_size, len(model_dims["num_controls"])), dtype=jnp.uint8)
obs, task = task.step(keys, actions=actions)

task.reset(jr.split(keys[-1], batch_size))

([Array([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [

In [16]:
num_params = 4 # two likelihood contingency params (reward prob, punishment prob), outcome preference param, prior state prob param
num_agents = batch_size
prior = Normal(num_params, num_agents, backend="numpyro")

def transform(z):
    na, np = z.shape

    reward_prob = nn.sigmoid(z[..., 0])
    punish_prob = nn.sigmoid(z[..., 1])
    lam = nn.softplus(z[..., 2])
    d = nn.sigmoid(z[..., 3])

    A = lax.stop_gradient(task.params["A"])
    B = lax.stop_gradient(task.params["B"])

    zeros = jnp.zeros_like(reward_prob)

    side = jnp.broadcast_to(jnp.array([[1., 1.], [0., 0.], [0., 0.]]), (na, 3, 2))
    left_col = jnp.stack([
        jnp.stack([1 - reward_prob, 1 - punish_prob], -1),
        jnp.stack([reward_prob, zeros], -1),
        jnp.stack([zeros, punish_prob], -1),
    ], -2)
    right_col = jnp.stack([
        jnp.stack([1 - punish_prob, 1 - reward_prob], -1),
        jnp.stack([zeros, reward_prob], -1),
        jnp.stack([punish_prob, zeros], -1),
    ], -2)
    A[1] = jnp.stack([side, left_col, right_col, side, side], axis=-2)

    C = [
        jnp.zeros((na, A[0].shape[1])),
        jnp.expand_dims(lam, -1) * jnp.array([0., 1., -1.]),
        jnp.zeros((na, A[2].shape[1])),
    ]
    D = [
        jnp.zeros((na, B[0].shape[1])).at[:, 0].set(1.0),
        jnp.stack([d, 1 - d], -1),
    ]

    return Agent(
        A,
        B,
        C=C,
        D=D,
        A_dependencies=A_dependencies,
        B_dependencies=B_dependencies,
        batch_size=na,
    )


key, _key = jr.split(seed_key)
z = jr.normal(_key, shape=(num_agents, num_params)) / 10

agents = transform(z)

In [17]:
from pybefit.inference.methods import default_dict_nuts

opts_task = {
    "task": task,
    "num_blocks": num_blocks,
    "num_trials": num_timesteps,
    "num_agents": num_agents,
}
opts_model = {"prior": {}, "transform": {}, "likelihood": opts_task}

model = NumpyroModel(prior, transform, likelihood, opts=opts_model)

pred = Predictive(model, num_samples=1)
key, _key = jr.split(key)
samples = pred(_key)

ValueError: vmap got inconsistent sizes for array axes to be mapped:
  * most axes (12 of them) had size 100, e.g. axis 0 of argument self.params['A'][0] of type float32[100,5,5];
  * one axis had size 2: axis 0 of argument key of type uint32[2]